In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

In [2]:

# Read in connectomes
test_connectome = pd.read_csv('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/widsdatathon2025/TEST/TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv')
train_connectome = pd.read_csv('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/widsdatathon2025/TRAIN_NEW/TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv')

# Read in solutions 
solutions = pd.read_excel('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/widsdatathon2025/TRAIN_NEW/TRAINING_SOLUTIONS.xlsx')



In [3]:
# Read in survey data
train_cat_quant = pd.read_csv('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/Preprocessing/train_cat_quant_imputed.csv')
test_cat_quant = pd.read_csv('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/Preprocessing/test_cat_quant_imputed.csv')

In [4]:
train_connectome.head(1)

,participant_id,0throw_1thcolumn,0throw_2thcolumn,0throw_3thcolumn,0throw_4thcolumn,0throw_5thcolumn,0throw_6thcolumn,0throw_7thcolumn,0throw_8thcolumn,0throw_9thcolumn,...,195throw_196thcolumn,195throw_197thcolumn,195throw_198thcolumn,195throw_199thcolumn,196throw_197thcolumn,196throw_198thcolumn,196throw_199thcolumn,197throw_198thcolumn,197throw_199thcolumn,198throw_199thcolumn
0,70z8Q2xdTXM3,0.22293,0.527903,0.429966,0.060457,0.566489,0.315342,0.508408,-0.07829,0.525692,...,0.224985,0.397448,0.422966,0.184642,0.305549,0.420349,0.016328,0.561864,0.47117,0.365221


In [ ]:
# Need to define the NonGraphDAE again to load trained model 

# Define NonGraphDAE class
class NonGraphDAE(nn.Module):
    def __init__(self, input_dim=19900, hidden_dim=512, latent_dim=128, dropout=0.2):
        super(NonGraphDAE, self).__init__()
        self.dropout = dropout

        # Encoder layers
        self.enc1 = nn.Linear(input_dim, hidden_dim)
        self.enc2 = nn.Linear(hidden_dim, latent_dim)
        
        # Decoder layers
        self.dec1 = nn.Linear(latent_dim, hidden_dim) 
        self.dec2 = nn.Linear(hidden_dim, input_dim)

        # Activation functions
        self.relu = nn.ReLU()
    
    def encode(self, x):
        x = self.relu(self.enc1(x))
        x = self.enc2(x)
        return x
    
    def decode(self, z):
        x = self.relu(self.dec1(z))
        x = self.dec2(x)
        return x
    
    def forward(self, x):
        z = self.encode(x)
        recon_x = self.decode(z)
        return recon_x, z

In [14]:
# Connectome data 
ids = train_connectome['participant_id']
connectome_features = train_connectome.iloc[:, 1:].values

# Convert to tensor
X = torch.tensor(connectome_features, dtype=torch.float32)

# Load autoencoder
autoencoder = torch.load('/Users/rubyc/Desktop/Datathon/WIDS_Datathon2025_Team/Archive/Models/nongraph_autoencoder.pth', weights_only=False)

# Set to evaluation mode
autoencoder.eval()

# Get latent reps
with torch.no_grad():
    _, z = autoencoder(X)  # z is the latent embeddings
    encoded_features = z.numpy()

# Save encoded features with IDs
encoded_df = pd.DataFrame(encoded_features, columns=[f"latent_{i}" for i in range(encoded_features.shape[1])])
encoded_df["participant_id"] = ids

In [ ]:
merged_encoded_df = encoded_df.merge(train_cat_quant, on='participant_id', how='left')

# merged_encoded_df.to_csv('merged_encoded_data.csv')